In [1]:
# geo libraries
import geopandas as gpd
import pandas as pd
import pystac_client
import shapely
from shapely import geometry, wkt
import leafmap
import rasterio
from rasterio.plot import show
import rioxarray

# other libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from IPython.display import JSON

In [2]:
import pyproj
pyproj.datadir.get_data_dir()

'/home/apolo/anaconda3/envs/geospatial/lib/python3.11/site-packages/pyproj/proj_dir/share/proj'

In [3]:
coords = [
    [-102.00723310488662,40.596123257503024],
    [-102.00723310488662,40.58168585757733],
    [-101.9882214495914,40.58168585757733],
    [-101.9882214495914,40.596123257503024],
    [-102.00723310488662,40.596123257503024]
]

polgyon = shapely.Polygon(coords)

In [4]:
aoi_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polgyon])

In [29]:
Map = leafmap.Map(center=[40.596123257503024, -102.00723310488662], zoom=13)
Map.add_basemap("USGS NAIP Imagery")
Map.add_gdf(aoi_gdf, layer_name="test", style={"color": "yellow", "fillOpacity": 0.1, "clickable": True})

In [30]:
Map

Map(center=[40.596123257503024, -102.00723310488662], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [6]:
def search_sentinel2_collection(start_date, end_date, aoi_geometry, max_cloud=100):
    """
    Search Sentinel 2 data collection for target_date
    and collect results including meta data in a dictionary.
    This function uses the PySTAC client
    """
    client = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")
    collection = "sentinel-2-l2a"
    
    search = client.search(
        collections=[collection],
        query = {"eo:cloud_cover": {"lt": max_cloud}},
        intersects=aoi_geometry.to_crs("EPSG:4326").geometry[0].__geo_interface__,
        datetime=f"{start_date}/{end_date}"
    )
    
    s2_items = []
    for item in search.items_as_dicts():
        s2_items.append(item)
        
    return s2_items

In [8]:
# search parameters
analysis_start_date=pd.to_datetime("2024-07-01T00:00:00Z").date()
analysis_end_date=pd.to_datetime("2024-07-31T00:00:00Z").date()
aoi_geometry = aoi_gdf.geometry
max_cloud=5

In [31]:
aoi_gdf

,geometry
0,"POLYGON ((-102.00723 40.59612, -102.00723 40.5..."


In [35]:
aoi_gdf.geometry.to_crs("EPSG:4326").geometry

0    POLYGON ((-102.00723 40.59612, -102.00723 40.5...
Name: geometry, dtype: geometry

In [24]:
aoi_gdf.geometry.to_crs("EPSG:4326").geometry[0].__geo_interface__

{'type': 'Polygon',
 'coordinates': (((-102.00723310488662, 40.596123257503024),
   (-102.00723310488662, 40.58168585757733),
   (-101.9882214495914, 40.58168585757733),
   (-101.9882214495914, 40.596123257503024),
   (-102.00723310488662, 40.596123257503024)),)}

In [8]:
sentinel2_items = search_sentinel2_collection(
    start_date=analysis_start_date,
    end_date=analysis_end_date,
    aoi_geometry = aoi_geometry,
    max_cloud=max_cloud
)

In [ ]:
JSON(sentinel2_items)

In [ ]:
thumbnail_path = sentinel2_items[-1]["assets"]["thumbnail"]["href"]
print(thumbnail_path)

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
    src_thumbnail = rasterio.open(thumbnail_path)


fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
show(src_thumbnail, ax=ax)
plt.show()